In [13]:
import sqlite3
import pandas as pd

In [14]:
db_connection=sqlite3.connect('shonen_jump.sqlite3')
cursor=db_connection.cursor()

In [15]:
cursor.execute("""SELECT MIN(release_date), MAX(release_date) FROM chapters;""")

In [16]:
(earliest, latest)=cursor.fetchone()
print(earliest)
print(latest)

2022-01-16
2025-11-09


In [17]:
df=pd.read_sql_query("""                     
                    SELECT chapters.series, 
                    COALESCE(debuts.release_date,?) AS start, 
                    COALESCE(finales.release_date,?) AS stop 
                    FROM chapters LEFT JOIN debuts ON chapters.series=debuts.series
                    LEFT JOIN finales ON chapters.series=finales.series
                    GROUP BY chapters.series;
                    """,params=(earliest,latest), con=db_connection)

In [18]:
df

,series,start,stop
0,Akane-banashi,2022-02-13,2025-11-09
1,Aliens Area,2022-06-05,2022-10-23
2,Astro Royale,2024-04-14,2025-04-20
3,Ayakashi Triangle,2022-01-16,2022-01-16
4,Ayashimon,2022-01-16,2022-05-29
...,...,...,...
64,Two on Ice,2023-09-24,2024-04-14
65,Ultimate Exorcist Kiyoshi,2024-06-23,2025-11-09
66,Undead Unluck,2022-01-16,2025-01-26
67,WITCH WATCH,2022-01-16,2025-11-09


In [ ]:
# We get a table with series as well as their earliest and latest (non-one-shot) chapters recorded in the database
df=pd.read_sql_query("""                     
                SELECT series, MIN(release_date) AS start, 
                MAX(release_date) AS stop FROM chapters 
                WHERE type!='One-Shot' GROUP BY series 
                """,con=db_connection)
# We convert this dataframe to a list of dictionaries of the form: 
# {'series': SERIES_NAME, 'start': START_DATE_STRING, 'stop': STOP_DATE_STRING}
series_info=df.to_dict(orient='records')
cursor.executemany("""
INSERT OR IGNORE INTO absences(series, issue_date)
SELECT DISTINCT :series AS series_title, release_date FROM chapters 
WHERE release_date BETWEEN :start AND :stop 
EXCEPT SELECT DISTINCT :series AS series_title, release_date FROM chapters
WHERE series= :series;""",series_info)